In [40]:
import scipy.stats as st
import numpy as np

def cochranSS(confidence, win_rate, margin_of_error=.05):
    '''
    Calculate Cochran Sample Size of Statistical Significance
    n = Z**2 * p * q // e**2
    Z: ZScore of confidence level, percentile
    p: Any currently known constant of sample
    q: 1-p
    e: margin of errror
    '''
    z = st.norm.ppf(1-(1-confidence)/2) #Confirm == ~ 1.96
    assert round(st.norm.ppf(1-(1-.95)/2),2) == 1.96
    num = (z**2) * (win_rate) * (1 - win_rate)
    den = margin_of_error ** 2
    return np.ceil(num / den)


cochranSS(.85, .6)

for i in np.linspace(.7, .95, 6):
    print(f'confidence level {round(i,2)} -- {cochranSS(i,.6)}')
    
def getConfidenceRange(beg, end, step, win_rate = .6):
    n_steps = ((end - beg) / step) + 1
    conf_lvls = {}
    for c in np.linspace(beg, end, int(n_steps)):
        conf_lvls[round(c,2)] = cochranSS(c, win_rate)
    return conf_lvls
        

'''Mini Test...'''
conf = getConfidenceRange(.7,1,.05)
assert conf[.95] == cochranSS(.95, .6), 'check your getConfidenceRange function'

confidence level 0.7 -- 104.0
confidence level 0.75 -- 128.0
confidence level 0.8 -- 158.0
confidence level 0.85 -- 199.0
confidence level 0.9 -- 260.0
confidence level 0.95 -- 369.0


In [5]:
loop = [print(f'confidence level {round(conf,2)} -- {cochranSS(conf,.6)}') for conf in np.linspace(.5,.75, 6)]

confidence level 0.5 -- 44.0
confidence level 0.55 -- 55.0
confidence level 0.6 -- 68.0
confidence level 0.65 -- 84.0
confidence level 0.7 -- 104.0
confidence level 0.75 -- 128.0


In [42]:
'''Test driven development'''


def testConfidenceRangeandCSS(beg=.7, end=.95, step=.05, win_rate=.6):
    c = getConfidenceRange(beg, end, step, win_rate)
    for conf, value in c.items():
        print(value, cochranSS(conf, win_rate))
        assert value == cochranSS(conf,win_rate), 'Error!'
    print('Success.')
        
    
testConfidenceRangeandCSS()

104.0 104.0
128.0 128.0
158.0 158.0
199.0 199.0
260.0 260.0
369.0 369.0
Success.


In [57]:
FUTs = {'NQ':16.5, 'ES':13.2,'GC':9,'RTY':6.38,'CL':7.5,'KC':4.5, 'SF':4.4,'JY':4, 'EC':2.5,'NG':2.2}

#futs = [('NQ',16.5),('ES',13.2),('GC',9),('RTY',6.38),('CL', 7.5),('KC,4.5'),('SF',4.4),('JY',4),('EC',2.5),('NG',2.2)]

f_srt = sorted(list(FUTs.items()), key=lambda x: x[1])

acct_bal = 39.580
init = acct_bal
finals = []
for sym, val in sorted(list(FUTs.items()), key=lambda x: x[1]):
    if acct_bal - val > 0:
        acct_bal -= val
        finals.append(sym)
    else:
        print('Stopped -- Margin Met',sym)
        break
print(f'$ Used -- {init - acct_bal}')
print(f'Remaining Account -- {acct_bal}')
print(f'Final Symbols -- {finals}')
    
finals  

Stopped -- Margin Met GC
$ Used -- 31.48
Remaining Account -- 8.099999999999998
Final Symbols -- ['NG', 'EC', 'JY', 'SF', 'KC', 'RTY', 'CL']


['NG', 'EC', 'JY', 'SF', 'KC', 'RTY', 'CL']

In [116]:

def getInitMargin(margin_dict, init_acct, cushion=.10, asc=False, skip=0, sym_skip=[]):
    balance = init_acct
    min_bal = init_acct * cushion if cushion > 0 else 0
    final = []
    for sym, val in sorted(list(margin_dict.items()), key=lambda x: x[1], reverse=asc)[skip:]:
        if sym in sym_skip:
            continue
        if balance - val > min_bal:
            balance -= val
            final.append(sym)
        else:
            print(f'Min Margin met -- {sym}')
            break
    print(f'$ Used -- {init_acct - balance}')
    print(f'$ Not Utilized -- {balance - min_bal}')
    print(f'Margin Cushion -- {balance}')
    print(f'Final Symbols -- {final}')
    return final

FUTs = {'NQ':16.5, 'ES':13.2,'GC':9,'RTY':6.38,'CL':7.5,'KC':4.5, 'SF':4.4,'JY':4, 'EC':2.5,'NG':2.2}
getInitMargin(FUTs,39,0,True,0,['NQ','SF','CL','JY','KC']) #'GC','NG',

$ Used -- 33.28
$ Not Utilized -- 5.7200000000000015
Margin Cushion -- 5.7200000000000015
Final Symbols -- ['ES', 'GC', 'RTY', 'EC', 'NG']


['ES', 'GC', 'RTY', 'EC', 'NG']

In [125]:
def getInitMargin_ID(margin_dict, init_acct, cushion=.10, asc=False, skip=0, sym_skip=[], ID_only=[]):
    balance = init_acct
    min_bal = init_acct * cushion if cushion > 0 else 0
    final = []
    ID = [] #Intraday only
    for sym, val in sorted(list(margin_dict.items()), key=lambda x: x[1], reverse=asc)[skip:]:
        if sym in sym_skip:
            continue
        if balance - val > min_bal:
            if sym in ID_only:
                balance -= val * .5
                #final.append(sym)
                ID.append(sym)
            else:
                balance -= val
                final.append(sym)
        else:
            print(f'Min Margin met -- {sym}')
            break
    print(f'$ Used  -- {init_acct - balance}')
    print(f'$ Not Utilized -- {balance - min_bal}') #Excess Cushion
    print(f'Margin Cushion -- {balance}')
    print(f'Final Symbols -- {final}')
    return final, ID

FUTs = {'NQ':16.5, 'ES':13.2,'GC':9,'RTY':6.38,'CL':7.5,'KC':4.5, 'SF':4.4,'JY':4, 'EC':2.5,'NG':2.2}
getInitMargin_ID(FUTs, 39, 0, True, sym_skip = ['SF','JY','KC','CL','GC'], ID_only=['NQ'])

$ Used -- 32.53
$ Not Utilized (excess cushion) -- 6.4700000000000015
Margin Cushion -- 6.4700000000000015
Final Symbols -- ['ES', 'RTY', 'EC', 'NG']


(['ES', 'RTY', 'EC', 'NG'], ['NQ'])

In [106]:
init = 39
total = 0
for sym, val in FUTs.items():
    if sym not in ['NQ','CL']:
        print(sym, val)
        init -= val
        if init <= min

ES 13.2
GC 9
RTY 6.38
KC 4.5
SF 4.4
JY 4
EC 2.5
NG 2.2


In [270]:
class InitialMarginOptimizer:
    
    '''TRY CHANGING TO SELF._FUTS (private) -- Consider adding cushion ?'''
    def __init__(self,initial_balance_thousands, futures_dict=None):
        self.init_bal = initial_balance_thousands
        if futures_dict is None:
            self.FUTs = {'NQ':16.5, 'ES':13.2,'GC':9,'RTY':6.38,'CL':7.5,'KC':4.5, 'SF':4.4,'JY':4, 'EC':2.5,'NG':2.2}
        else:
            self.FUTs = futures_dict
        #self.futures_dict = 0
        self.final = []
        self.intraday = []
        print(f'Initial Margin Optimizer Initialized -- Initial Balance -- {self.init_bal} \nFutures Dict -- {self.FUTs}')         
    
    @property      
    def futures_dict(self):
        return self.FUTs
    
    @futures_dict.setter
    def futures_dict(self,new):
        assert isinstance(new, tuple) or isinstance(new, dict)
        if isinstance(new, tuple) and len(new) == 2:
            self.FUTs[new[0]] = new[1]
        else:
            self.FUTs = new
        #tmp = sorted(self.FUTs.items(), key=lambda x: x[1], reverse=True)
        tmp = {}
        #Alot of effort -- maybe easier to use list of tuples?
        self.FUTs = {k:v for k,v in sorted(self.FUTs.items(), key=lambda x: x[1], reverse=True)}
        
            
    @property
    def initial_balance(self):
        return self.init_bal
    
    @initial_balance.setter
    def initial_balance(self, new_bal):
        if new_bal < 0:
            raise ValueError("inital balance < 0 is not possible.")
        self.init_bal = new_bal
    
        
    def Optimize(self, sym_skip = [], intraday_only = [], cushion = .1, asc = False, idx_skip = 0):
        balance = self.init_bal
        min_bal = self.init_bal * cushion if cushion > 0 else 0
        final, ID = [], []
        for sym, val in sorted(list(self.FUTs.items()), key=lambda x: x[1], reverse=asc)[idx_skip:]:
            if sym in sym_skip:
                continue
            if sym in intraday_only: #Intraday Margin -- 50% Discount
                print(sym, val)
                val *= .5
                print(sym, val)
            if balance - val > min_bal:
                if sym in intraday_only:
                    print(sym, val)
                    balance -= val
                    ID.append(sym)
                else:
                    balance -= val
                    final.append(sym)
            else:
                print(f'min margin met... {balance}')
                break

        print(f'$ Used  -- {self.init_bal - balance}')
        if balance - min_bal != balance:
            print(f'$ Excess Cushion -- {balance - min_bal}') #Excess Cushion
        print(f'$ Total Cushion -- {balance}')
        print(f'Final Symbols -- {final}')
        
        self.intraday = ID
        self.final = final
        return self.final, self.intraday
                
        


In [283]:
#Testing ...
import pytest


imo = InitialMarginOptimizer(39)


def test_dict_replace_and_add(new_dict, new_pair):
    #Initialize...
    imo = InitialMarginOptimizer(40)
    imo.futures_dict = {'NQ':20}
    
    #Test replace...
    imo.futures_dict = new_dict
    assert imo.futures_dict == pytest.approx(new_dict) , 'Replace -- Tuple Arg -- not working.'
    print(imo.futures_dict,'==',new_dict)

    #Test 'Append'
    imo.futures_dict = new_pair

    assert list(imo.futures_dict.keys())[1] == new_pair[0]
    assert len(imo.futures_dict) == 2
    
    #Test sort...
    a = sorted(imo.futures_dict.items(),key=lambda x: x[1], reverse=True)
    b = list(imo.futures_dict.items())
    assert a == b , 'Dict not sorting...'

    
    


def test_init_margin_property(new_value):
    imo = InitialMarginOptimizer(40)
    beg = imo.initial_balance
    #Test get / set
    imo.initial_balance = 0
    assert imo.initial_balance == 0
    
    #Test Exception / Error handling
    #import mymod #-- Not installed? 
    #self.assertRaises(ExpectedException, afunction, arg1, arg2)
    try:
        imo.initial_balance = -50
    except ValueError as e:
        print('Bingo ! Value error caught -- ',e)
        
    #Test revert back to normal, and both public and private methods work.
    imo.initial_balance = beg
    assert imo.initial_balance == beg
    assert imo.init_bal == beg
    
    
    
def test_optimize():
    imo = InitialMarginOptimizer(40)
    print(imo.futures_dict, imo.FUTs)
    assert len(imo.futures_dict) == 10
    
    #Test Optimize Function -- Descending + Intraday symbols
    f, ido = imo.Optimize(['KC','CL'],['NQ'],cushion = 0.1, asc = True)
    print(f,'==',['ES','GC'])
    assert f == ['ES','GC'], 'Optimize Desc Works -- Final Non-ID'
    assert ido == ['NQ'], 'Optimize Desk + ID Works'

    #Test Optimize Function -- Ascending without Intraday
    f, ido = imo.Optimize(['CL','GC','SF'],cushion = 0, asc = False)

    print(f,'==',['NG', 'EC', 'JY', 'KC', 'RTY', 'ES'])
    assert f == ['NG', 'EC', 'JY', 'KC', 'RTY', 'ES'], 'Optimize Asc works...'
    assert ido == []
    
    #Test 
    
    

    
#Testing Functions...


test_dict_replace_and_add({'NQ':18},('RB',9))
test_init_margin_property(20)
test_optimize()


Initial Margin Optimizer Initialized -- Initial Balance -- 39 
Futures Dict -- {'NQ': 16.5, 'ES': 13.2, 'GC': 9, 'RTY': 6.38, 'CL': 7.5, 'KC': 4.5, 'SF': 4.4, 'JY': 4, 'EC': 2.5, 'NG': 2.2}
Initial Margin Optimizer Initialized -- Initial Balance -- 40 
Futures Dict -- {'NQ': 16.5, 'ES': 13.2, 'GC': 9, 'RTY': 6.38, 'CL': 7.5, 'KC': 4.5, 'SF': 4.4, 'JY': 4, 'EC': 2.5, 'NG': 2.2}
{'NQ': 18} == {'NQ': 18}
Initial Margin Optimizer Initialized -- Initial Balance -- 40 
Futures Dict -- {'NQ': 16.5, 'ES': 13.2, 'GC': 9, 'RTY': 6.38, 'CL': 7.5, 'KC': 4.5, 'SF': 4.4, 'JY': 4, 'EC': 2.5, 'NG': 2.2}
Bingo ! Value error caught --  inital balance < 0 is not possible.
Initial Margin Optimizer Initialized -- Initial Balance -- 40 
Futures Dict -- {'NQ': 16.5, 'ES': 13.2, 'GC': 9, 'RTY': 6.38, 'CL': 7.5, 'KC': 4.5, 'SF': 4.4, 'JY': 4, 'EC': 2.5, 'NG': 2.2}
{'NQ': 16.5, 'ES': 13.2, 'GC': 9, 'RTY': 6.38, 'CL': 7.5, 'KC': 4.5, 'SF': 4.4, 'JY': 4, 'EC': 2.5, 'NG': 2.2} {'NQ': 16.5, 'ES': 13.2, 'GC': 9, 'RT

In [271]:
#imo.Optimize(39,FUTs)
imo = InitialMarginOptimizer(39,FUTs)

#imo.Optimize(sym_skip = ['KC','CL'],intraday_only=['NQ']) #sym_skip = [], intraday_only = [],



Initial Margin Optimizer Initialized -- Initial Balance -- 39 
Futures Dict -- {'NQ': 16.5, 'ES': 13.2, 'GC': 9, 'CL': 7.5, 'RTY': 6.38, 'KC': 4.5, 'SF': 4.4, 'JY': 4, 'EC': 2.5, 'NG': 2.2}


In [275]:
imo.Optimize(['KC','CL'],['NQ'],cushion = 0.1, asc = True)
imo.Optimize(['CL','GC','SF'],cushion = 0, asc = False)

NQ 16.5
NQ 8.25
NQ 8.25
min margin met... 8.55
$ Used  -- 30.45
$ Excess Cushion -- 4.65
$ Total Cushion -- 8.55
Final Symbols -- ['ES', 'GC']
min margin met... 6.219999999999999
$ Used  -- 32.78
$ Total Cushion -- 6.219999999999999
Final Symbols -- ['NG', 'EC', 'JY', 'KC', 'RTY', 'ES']


(['NG', 'EC', 'JY', 'KC', 'RTY', 'ES'], [])

In [264]:
def getInitMargin_ID(margin_dict, init_acct, cushion=.10, asc=False, skip=0, sym_skip=[], ID_only=[]):
    balance = init_acct
    min_bal = init_acct * cushion if cushion > 0 else 0
    final = []
    ID = [] #Intraday only
    for sym, val in sorted(list(margin_dict.items()), key=lambda x: x[1], reverse=asc)[skip:]:
        if sym in sym_skip:
            continue
            
        #Discount margin for intraday only
        if sym in ID_only: val *= .5
            
        #Check if we have $ for it...
        if balance - val > min_bal:
            if sym in ID_only:
                print(val)
                ID.append(sym)
            else:

                final.append(sym)
            balance -= val
        else:
            print(f'Min Margin met -- {sym}')
            break
    print(f'$ Used  -- {init_acct - balance}')
    print(f'$ Excess Cushion -- {balance - min_bal}') #Excess Cushion -- Beyond our stated cushion.
    print(f'$ Total Cushion -- {balance}')
    print(f'Final Symbols -- {final}')
    return final, ID

FUTs

getInitMargin_ID(FUTs, 39, 0.1, True, 0, ['KC','CL'],['NQ'])

8.25
Min Margin met -- RTY
$ Used  -- 30.45
$ Excess Cushion -- 4.65
$ Total Cushion -- 8.55
Final Symbols -- ['ES', 'GC']


(['ES', 'GC'], ['NQ'])